In [ ]:
import pymongo
import pandas as pd
from tqdm import tqdm

from notebook_modules.database import Database
from notebook_modules.half import Half
from notebook_modules.distribution import aggregate
from notebook_modules.lists import save_db, load_db

In [ ]:
load_from_fs = True

if not load_from_fs:
    db = Database()
    assert db.client, "No database client available!"
    stackoverflow = db.client["stackoverflow"]
    tags = stackoverflow["tags"]
    posts = stackoverflow["posts"]

In [ ]:
current_period = Half.make_half(2019, 2)
halves = Half.make_halves(2008, 2020)

In [ ]:
if load_from_fs:
    full_tags = load_db(topic="tsa", name="full-tags")
else:
    full_tags = aggregate(posts, current_period, full=True)
    save_db(full_tags, topic="tsa", name="full-tags")
full_tags

In [ ]:
def aggregate_time_periods(periods, unit="half"):
    time_periods = []

    for period in tqdm(periods, unit=unit, ascii=True):
        tags_from_posts = aggregate(posts, period)
        if tags_from_posts.empty:
            time_periods.append({"_date": period.end})
        else:
            keys = tags_from_posts.tag.values
            values = tags_from_posts.frequency.values
            time_periods.append({"_date": period.end, **dict(zip(keys, values))})

    return time_periods

In [ ]:
if load_from_fs:
    time_series = load_db(topic="tsa", name="time-series")
else:
    time_periods = aggregate_time_periods(halves, unit="half")
    time_series = pd.DataFrame(time_periods, columns=["_date", *full_tags.tag.values])
    time_series.fillna(0, inplace=True)
    save_db(time_series, topic="tsa", name="time-series")
time_series

In [ ]:
from keras.layers import Dense, LSTM, Dropout
from keras.models import Sequential
from keras.utils import plot_model

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

In [ ]:
scaler = MinMaxScaler(feature_range=(0, 1))
data = scaler.fit_transform(data)

In [ ]:
model = Sequential()

model.add(LSTM(units=50, return_sequences=True, input_shape=None)
model.add(Dropout(0.2))

model.add(LSTM(units=50, return_sequences=True))
model.add(Dropout(0.2))

model.add(LSTM(units=50, return_sequences=True))
model.add(Dropout(0.2))

model.add(LSTM(units=50))
model.add(Dropout(0.2))

model.add(Dense(units=1))

model.compile(optimizer="adam", loss="mean_squared_error")

model.fit(features_set, labels, epochs=100, batch_size=32)

In [ ]:
plot_model(model, show_shapes=True, show_layer_names=True, to_file="output/models/model-tsa.png")